In [ ]:
import pandas as pd
import folium 
import seaborn
import os
import json
# Used for downloading and handling files:
import gzip
import urllib
#import requests

In [ ]:
folium.__version__

# 01 - Unemployment in European countries



In [ ]:
# Downloading and reading in the data
#urllib.request.urlretrieve(
       # 'http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/tipsun20.tsv.gz', 
        #'tipsun20.tsv.gz')
file = gzip.open('tipsun20.tsv.gz')
df = pd.read_csv(file, sep='\t')

# Re-organising the columns

df = df.rename(columns={'sex,age,unit,geo\\time': 'country'})
df = df.set_index('country')
df = df.filter(like='TOTAL', axis=0) # filter out extra data, only interested in total unemployment rate
df.head()

In [ ]:
m_eu = folium.Map([51, 15], zoom_start=4)
eu_topo_path = r'topojson/europe.topojson.json'
folium.TopoJson(open(eu_topo_path),
                'objects.europe',
                name='topojson',
               ).add_to(m_eu)
m_eu

# 02 - Unemployment in Swiss cantons

In [ ]:
m_ch = folium.Map([47,8.3], zoom_start=8)
canton_topo_path = r'topojson/ch-cantons.topojson.json'
#m_ch.geo_json(geo_path=canton_topo_path, topojson='objects.cantons')
#m_ch.geo_json

folium.TopoJson(open(canton_topo_path),
                'objects.cantons',
                name='topojson',
               ).add_to(m_ch)
m_ch

In [ ]:
df.head()